# Visão Computacional - Encontrando as guias-Parte 3


## Transformada de HOUGH

A Transformada de Hough é um método padrão para detecção de formas que são facilmente parametrizadas (linhas, círculos, elipses, etc.) em imagens digitalizadas.

* A idéia é aplicar na imagem uma transformação tal que
todos os pontos pertencentes a uma mesma curva
sejam mapeados num único ponto de um novo espaço
de parametrização da curva procurada.

### Do começo...

A imagem a seguir demonstra o que acontece quando aplicamos a transformada de Hough. Nela é possivel verificar que quando aplicamos a tecnica a um segmento de reta encontramos 1 ponto no Espaço Hough.

<div , align = 'center'>
    <img src="Slide14.png" style="width:800px;height:500px;">
</div> 

Mas o que aconteceria se ao invés de uma reta aplicarmos um ponto com referência. Ora como sabemos da equação da reta é necessário no minimo dois pontos para criar uma reta. Então o que aconteceria se passasemos diversas retas em um ponto?

<div , align = 'center'>
    <img src="Slide15.png" style="width:800px;height:500px;">
</div> 

Com este conhecimento podemos entender que podemos ter diversos pontos que convergem para um valor de referência. Com isso podemos entender que para a construção de 1 reta é necessário de 2 pontos de referencia. Olhe para a figura a seguir :
<div , align = 'center'>
    <img src="Slide19.png" style="width:800px;height:500px;">
</div> 

Quando aplicamos a técnica para 3 pontos, podemos ver que no espaço Hough existe um ponto de convergência. Este ponto retorna para o leitor a angulação e o ponto que é cruzado pela reta no eixo y.

### Aplicando o conceito


Como sabemos, apos aplicarmos o `Canny()` temos diversas linhas parametrizadas

<div , align = 'center'>
    <img src="Slide10.png" style="width:800px;height:500px;">
</div> 

Depois disso aplicamos a Mascára:

<div , align = 'center'>
    <img src="Slide22.png" style="width:800px;height:500px;">
</div> 

Da mascara podemos, intuitivamente, identificar as linhas de tendência:
<div , align = 'center'>
    <img src="Slide21.png" style="width:800px;height:500px;">
</div>  

Vamos aplicar a transformação atribuindo um valor para o valor em uma grade que mais proporcione uma reta para os dados da mascará. A seguir uma demostração generica de como aplicaremos o conceito

<div , align = 'center'>
    <img src="Slide23.png" style="width:800px;height:500px;">
</div>  

### Então vamos programar ... Não tão rapido

Apesar de ser um ótimo método, a trasformada, da maneira que estamos utilizando, não consegue realizar-se para pontos que esteja na vertical. Uma vez que m é a representação da tangente da reta, assim como os pontos estaria alinhados, a variação de $x$ seria $\infty$.

$$m = \frac{\Delta y}{\Delta x}$$

Mas se trocaramos os parâmetros de entrada? 

Ao invés de:

$$y = m*x + b$$

utilizar-se-mos as coordenadas polares:

$$\rho = x*cos(\Theta ) + y*sin(\Theta)$$



<div , align = 'center'>
    <img src="Slide24.png" style="width:800px;height:500px;">
</div>  

<div , align = 'center'>
    <img src="Slide25.png" style="width:800px;height:500px;">
</div>  

### Ao codigo 

In [4]:
import cv2 
import numpy as np


def canny(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    canny = cv2.Canny(blur,50,150)
    return canny

def regiao_de_interresse(image):
    altura = image.shape[0]                                           
    triangulo = np.array([[(220,altura), (1100,altura),(550,250)]]) 
    mask = np.zeros_like(image)         
    cv2.fillPoly(mask, triangulo, 255) 
    mask_image = cv2.bitwise_and(image,mask)
    return mask_image

image = cv2.imread('test_image.jpg')
image_estrada = np.copy(image)
canny_image = canny(image_estrada)
cropado = regiao_de_interresse(canny)

Aplicando a transformada. 

* Note que estamos passando os argumento e restruturando em uma imagem completamente preta e redimensionando um vetor  de 4 dimensoes em 4 variaveis, para criar as linhas necessária para criar os segmentos de reta.

In [6]:
def display_lines(image,lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            cv2.line(line_image, (x1,y1),(x2,y2),(255,0,0),10)
    return line_image            

lines = cv2.HoughLinesP(cropado, 2 ,np.pi/180, 100 ,np.array([]), minLineLength = 40 , maxLineGap = 5)

line_image = display_lines(image_estrada,lines)



In [9]:
cv2.imshow("resultado",line_image)
cv2.waitKey(0)

-1

<div , align = 'center'>
    <img src="Slide26.png" style="width:800px;height:500px;">
</div>  

### Unindo as imagens:

Vamos sobrepor a imagem copiada com 80% da saturação com a imagem das linhas com 100% de saturação para que fique mais facil de enxegar a imagem final

In [10]:
imagens_unidas = cv2.addWeighted(image_estrada, 0.8 , line_image , 1 , 1)
cv2.imshow("resultado",imagens_unidas)
cv2.waitKey(0)

-1

<div , align = 'center'>
    <img src="resultado.png" style="width:800px;height:500px;">
</div>